# BayesDB analysis of fish bouts in relation to paramecia characteristics

## 1. Ingesting the data

### a. Launching BayesDB

In [54]:
%load_ext jupyter_probcomp.magics

The jupyter_probcomp.magics extension is already loaded. To reload it, use:
  %reload_ext jupyter_probcomp.magics


In [55]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

In [56]:
!rm -f 042318_6/bdb_hunts_inverted.bdb
%bayesdb -j 042318_6/bdb_hunts_inverted.bdb

u'Loaded: 042318_6/bdb_hunts_inverted.bdb'

### b. Loading and printing the data from a `.csv` file

In [57]:
%sql DROP TABLE IF EXISTS bout_table;
%bql CREATE TABLE bout_table FROM '042318_6/huntbouts_inverted.csv' 

""


In [58]:
%sql SELECT * FROM bout_table LIMIT 25;

,Bout Number,Bout Az,Bout Alt,Bout Dist,Bout Delta Pitch,Bout Delta Yaw,Para Az,Para Alt,Para Dist,Para Az Velocity,Para Alt Velocity,Para Dist velocity,Postbout Para Az,Postbout Para Alt,Postbout Para Dist,Strike Or Abort
0,0.0,1.329745,0.368559,193.861211,0.210165,0.799354,1.429832,0.516095,352.929936,-0.007099,-0.001299,-1.457911,0.651804,0.649091,175.322735,1.0
1,1.0,1.982416,0.299406,55.543836,0.066884,0.494285,0.637470,0.632546,173.949790,0.004821,-0.004269,-1.182665,-0.156973,0.444844,161.943651,1.0
2,0.0,0.506706,-0.059321,53.007906,-0.133398,0.290902,0.323264,0.189230,281.018411,-0.055263,-0.004879,-4.543830,-0.085878,0.306153,242.040221,2.0
3,1.0,0.600239,0.174290,41.940722,0.036709,0.168698,0.084501,0.328278,243.871697,-0.002494,-0.000956,0.474774,0.129235,0.276962,214.811805,2.0
4,2.0,0.011075,0.067245,51.069587,-0.013290,0.006640,0.123433,0.263269,212.646078,-0.001821,-0.004629,-0.596787,0.116984,0.249567,168.922531,2.0
5,3.0,0.371120,-0.001310,67.708218,0.016486,0.165904,0.117435,0.245743,167.145417,0.001698,-0.001300,-2.282867,-0.296094,0.167755,120.486096,2.0
6,4.0,0.401825,-0.088805,75.532485,-0.016285,0.205076,0.302955,0.094228,113.713507,-0.007594,-0.018352,-0.128023,0.016037,-0.016300,68.445575,2.0
7,0.0,1.744683,0.078227,70.882601,0.145857,0.555345,0.959816,0.405993,250.547543,-0.008977,0.002937,-1.182338,-0.098080,0.428425,206.296536,1.0
8,1.0,0.226221,0.339171,37.712165,0.045224,0.077200,0.088988,0.426451,206.439055,-0.008272,-0.001612,-0.082692,0.019877,0.384856,178.601206,1.0
9,2.0,0.188983,0.384293,31.361674,0.021267,0.059719,0.021460,0.384433,177.920890,0.001165,0.000561,-1.093454,-0.034940,0.368936,157.546408,1.0


In [59]:
%bql .nullify bout_table ''
%bql .nullify bout_table 'nan'

Nullified 0 cells
Nullified 0 cells


## 2. Automatically learning a CrossCat probabilistic model

### a. Defining an analysis population

In [60]:
%bql GUESS SCHEMA FOR bout_table

,column,stattype,num_distinct,reason
0,Bout Number,nominal,9.0,There are fewer than 20 distinct numerical va...
1,Bout Az,numerical,59.0,There are at least 20 unique numerical values...
2,Bout Alt,numerical,59.0,There are at least 20 unique numerical values...
3,Bout Dist,numerical,59.0,There are at least 20 unique numerical values...
4,Bout Delta Pitch,numerical,59.0,There are at least 20 unique numerical values...
5,Bout Delta Yaw,numerical,59.0,There are at least 20 unique numerical values...
6,Para Az,numerical,59.0,There are at least 20 unique numerical values...
7,Para Alt,numerical,59.0,There are at least 20 unique numerical values...
8,Para Dist,numerical,59.0,There are at least 20 unique numerical values...
9,Para Az Velocity,numerical,59.0,There are at least 20 unique numerical values...


In [61]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Bout Number";)

""


### b. Creating and analyzing a probabilistic model (automatically)

In [62]:
%mml CREATE GENERATOR huntbouts_crosscat FOR bout_population;

""


In [63]:
%mml INITIALIZE 100 MODELS IF NOT EXISTS FOR huntbouts_crosscat;

""


In [64]:
%mml ANALYZE huntbouts_crosscat FOR 100 ITERATIONS (OPTIMIZED);
# note you can use the (OPTIMIZED) flag here but that gave you weird results last time. 

Completed: 100 iterations in 3.318850 seconds.
Completed: 100 iterations in 3.383471 seconds.
Completed: 100 iterations in 3.123108 seconds.
Completed: 100 iterations in 3.874600 seconds.
Completed: 100 iterations in 3.329184 seconds.
Completed: 100 iterations in 3.403007 seconds.
Completed: 100 iterations in 2.939532 seconds.
Completed: 100 iterations in 3.408313 seconds.
Completed: 100 iterations in 3.283163 seconds.
Completed: 100 iterations in 3.729219 seconds.
Completed: 100 iterations in 3.291378 seconds.
Completed: 100 iterations in 2.257562 seconds.
Completed: 100 iterations in 4.780823 seconds.
Completed: 100 iterations in 2.569543 seconds.
Completed: 100 iterations in 4.840901 seconds.
Completed: 100 iterations in 3.163851 seconds.
Completed: 100 iterations in 2.596051 seconds.
Completed: 100 iterations in 2.343927 seconds.
Completed: 100 iterations in 4.613838 seconds.
Completed: 100 iterations in 2.464253 seconds.
Completed: 100 iterations in 3.751396 seconds.
Completed: 10

""
